In [1]:
import pandas as pd 
import pyodbc
import json

In [2]:
df = pd.read_csv('Popular_Baby_Names.csv')
df

,Year of Birth,Gender,Ethnicity,Child's First Name,Count,Rank
0,2011,FEMALE,HISPANIC,GERALDINE,13,75
1,2011,FEMALE,HISPANIC,GIA,21,67
2,2011,FEMALE,HISPANIC,GIANNA,49,42
3,2011,FEMALE,HISPANIC,GISELLE,38,51
4,2011,FEMALE,HISPANIC,GRACE,36,53
...,...,...,...,...,...,...
57577,2014,MALE,WHITE NON HISPANIC,Yousef,18,94
57578,2014,MALE,WHITE NON HISPANIC,Youssef,24,88
57579,2014,MALE,WHITE NON HISPANIC,Yusuf,16,96
57580,2014,MALE,WHITE NON HISPANIC,Zachary,90,39


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57582 entries, 0 to 57581
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Year of Birth       57582 non-null  int64 
 1   Gender              57582 non-null  object
 2   Ethnicity           57582 non-null  object
 3   Child's First Name  57582 non-null  object
 4   Count               57582 non-null  int64 
 5   Rank                57582 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 2.6+ MB


In [4]:
# Mélanger les lignes pour une répartition aléatoire des données
df = df.sample(frac=1).reset_index(drop=True)

In [5]:
df

,Year of Birth,Gender,Ethnicity,Child's First Name,Count,Rank
0,2014,MALE,ASIAN AND PACIFIC ISLANDER,Jeffrey,17,51
1,2013,MALE,ASIAN AND PACIFIC ISLANDER,Jordan,19,48
2,2014,MALE,HISPANIC,Kai,13,95
3,2012,MALE,HISPANIC,ISAIAH,79,43
4,2013,FEMALE,WHITE NON HISPANIC,Ester,10,82
...,...,...,...,...,...,...
57577,2011,FEMALE,HISPANIC,AMY,87,17
57578,2012,MALE,HISPANIC,JORDAN,57,58
57579,2013,MALE,WHITE NON HISPANIC,Meir,40,70
57580,2013,FEMALE,HISPANIC,Kendra,17,71


In [6]:
# Calculer le nombre de lignes pour chaque partie
total_rows = len(df)
json_rows = int(0.3 * total_rows)
db_rows = int(0.3 * total_rows)
csv_rows = total_rows - json_rows - db_rows

In [7]:
# Diviser les données en trois parties
json_data = df[:json_rows]
db_data = df[json_rows:json_rows + db_rows]
csv_data = df[json_rows + db_rows:]

In [8]:
# Sauvegarder les données au format JSON
json_data.to_json("donnees.json", orient="records", lines=True)


In [9]:
# Se connecter à la base de données SQL Server
conn = pyodbc.connect("Driver={SQL Server};Server=LAPTOP-IGO4LBSV\SQLEXPRESS;Database=Popular_Baby_Names;Trusted_Connection=yes;")
cursor = conn.cursor()

cursor.execute("SELECT @@version;")
row = cursor.fetchone()
while row: 
    print(row[0])
    row = cursor.fetchone()


Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) 
	Oct  8 2022 05:58:25 
	Copyright (C) 2022 Microsoft Corporation
	Express Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 19045: )



In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57582 entries, 0 to 57581
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Year of Birth       57582 non-null  int64 
 1   Gender              57582 non-null  object
 2   Ethnicity           57582 non-null  object
 3   Child's First Name  57582 non-null  object
 4   Count               57582 non-null  int64 
 5   Rank                57582 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 2.6+ MB


In [ ]:
# Créer une table dans la base de données
cursor = conn.cursor()
cursor.execute('''create table Popular_Baby_Names (
               id INT PRIMARY KEY NOT NULL IDENTITY ,
               year int,
               Gender varchar(55),         
               Ethnicity varchar(100),
               FirstName varchar(100),
               Count int,
               Rank int          
              )
               ''')
cursor.commit()


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57582 entries, 0 to 57581
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Year of Birth       57582 non-null  int64 
 1   Gender              57582 non-null  object
 2   Ethnicity           57582 non-null  object
 3   Child's First Name  57582 non-null  object
 4   Count               57582 non-null  int64 
 5   Rank                57582 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 2.6+ MB


In [13]:
cursor = conn.cursor()

for index, row in db_data.iterrows():
        cursor.execute("INSERT INTO Popular_Baby_Names (year, Gender,Ethnicity,FirstName,Count,Rank) VALUES (?, ?, ?, ?, ?, ?)", row["Year of Birth"], row["Gender"], row["Ethnicity"], row["Child's First Name"], row["Count"], row["Rank"])

cursor.commit()
cursor.close()

In [14]:
# Sauvegarder les données au format CSV
csv_data.to_csv("donnees.csv", index=False)